# Modeling task 3

Данный ноутбук направлен на решение третьей задачи: поиска исходного трека в цепочке каверов.

Данную задачу будем частично решать аналитическим путём, частично при помощи ML. Для решения будем придерживаться следующего алгоритма:
1. при отсутствии цепочки каверов (модель из второй задачи не нашла похожие треки), будем считать сам трек оригиналом;
2. если есть цепочка треков, то получаем самый ранний из треков на основании года, полученному из isrc;
3. если несколько треков соответствуют самому раннему году, либо если есть треки, для которых год неизвестен, то отдаём их для прогноза модели, обученной для решения задачи классификации кавер / оригинал, и по predict_proba выбираем тот, в котором модель больше уверена.

Импортируем требуемые библиотеки и пути

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
import pickle
from typing import List

In [2]:
PATH_PREDICTION = 'data/preprocessing/prdeiction.csv'
PATH_TEST = 'data/preprocessing/test_task_1.csv'
PATH_TEST_TASK1_TRACK_ID = 'data/preprocessing/test_task_1_track_id.csv'
PATH_MODEL = 'models/task_1_model.pkl'

Откроем датасеты и выведем их размеры

In [3]:
df_pred = pd.read_csv(PATH_PREDICTION)
df_test = pd.read_csv(PATH_TEST)
df_test_track_id = pd.read_csv(PATH_TEST_TASK1_TRACK_ID)

In [5]:
df_pred.shape, df_test.shape, df_test_track_id.shape

((71399, 2), (14246, 34), (14246, 1))

df_pred естьественно больше, поскольку там прогноз для всего датасета, а в тесте только те, которые не видела модель классификации кавер / оригинал.

Выберем из датасета df_pred те треки track_id которых содержаться в df_test_track_id

In [6]:
df_pred = df_pred.merge(df_test_track_id, on='track_id', how='right')

In [7]:
df_pred.shape

(14246, 2)

Просоединим к df_test столбец с 'track_id' он нам понадобится для фильтрации данных

In [8]:
df_test = df_test.join(df_test_track_id)

Откроем модель

In [9]:
with open(PATH_MODEL, 'rb') as file:
    model = pickle.load(file)  

Напишем функцию, которая будет реализовывать предложенную выше логику действий. Для отметки оригинала добавим столбец original_track_id, в котором будем указывать предполагаемы track_id оригинала.

Но для работы данной функции нам потребуется вспомогательная функция, которая будет выбиратьболее ранний год и при необходимости использовать модель из 1 задачи.

In [10]:
def get_first_track(row, df_task_1, model):
    df = df_task_1.copy(deep=True)
    # составляем полную цепочку треков (основной + предсказанная цепочка)
    list_track = (row['track_id'] + ' ' + row['prediction_track_list']).split()
    # выбираем из переданного датасета только те, которые входят в list_track
    df = df[df['track_id'].isin(list_track)]
    # из полученного датасета оставляем только те, у которых минимальный год релиза или стоит метка об отсутствии isrc
    df = df[(df['num_year']==df['num_year'].min()) | (df['missing_isrc']==1)]
    # если получилась одна строка, то этот трек и является оригиналом
    if df.shape[0] == 1:
        return df['track_id'].values[0]
    
    
    # если получили больше 1 строки, то запускаем модель и добавляем к df столбец predict_proba
    df['predict_proba'] = model.predict_proba(df.drop('track_id', axis=1))[:,1]    
    # возвращаем track_id с максимальным значением predict_proba
    # добавляем i.loc, на случай, если вернётся несколько строк
    return df[df['predict_proba']==df['predict_proba'].max()].iloc[0]['track_id']

In [11]:
def get_original_track(df_task_2: pd.DataFrame, 
                       df_task_1:pd.DataFrame,
                       model: CatBoostClassifier) -> pd.DataFrame:
    df = df_task_2.copy(deep=True)
    df['original_track_id'] = np.nan
    mask = df['prediction_track_list'].isna()
    df.loc[mask, 'original_track_id'] = df.loc[mask, 'track_id']
    
    df.loc[~mask, 'original_track_id'] = df.loc[~mask].apply(get_first_track, 
                                                             df_task_1=df_task_1, 
                                                             model=model, 
                                                             axis=1)
    return df

In [12]:
df_pred_original = get_original_track(df_pred, df_test, model)

Выведем первые 5 строк

In [13]:
df_pred_original.head()

,track_id,prediction_track_list,original_track_id
0,f9ec63fb95c019c06e794eef1bedada5,NaN,f9ec63fb95c019c06e794eef1bedada5
1,3c3c15ccaf86e5e54723c8efd074944a,NaN,3c3c15ccaf86e5e54723c8efd074944a
2,418f8033b3c4d6d00aefb46585471673,NaN,418f8033b3c4d6d00aefb46585471673
3,38e9ef89e9d7fd0e0a6f770603a1f9ca,NaN,38e9ef89e9d7fd0e0a6f770603a1f9ca
4,10606a2a117354a2deaef2fde2c3c8f7,NaN,10606a2a117354a2deaef2fde2c3c8f7


Проверим наличие nan в original_track_id

In [14]:
df_pred_original['original_track_id'].isna().sum()

0

Выведем первые 5 строк датасета, где prediction_track_list не путой

In [15]:
df_pred_original[~df_pred_original['prediction_track_list'].isna()].head()

,track_id,prediction_track_list,original_track_id
140,cc2469cf94e93941afcdd26b30dbd398,02db8e8e8d3c08f3a57cc6db3876f942 255b537053a9e...,255b537053a9e1adbe9f76cd03669734
509,8176544797d18079e07b0130c80216cf,2e7110410d6c82206d88bf61cfa5cf14,8176544797d18079e07b0130c80216cf
908,7e9b19c0cfd247420f02d2128432fa5e,f24cb8af625fe6679b965b8e7561601e 0a2fbd20ecdb4...,7e9b19c0cfd247420f02d2128432fa5e
1082,5d58930596840ad631fe9cba17cee8d8,8b73d25bff03bb9a0b3c9b15e47c9688 61bc2470ec141...,5d58930596840ad631fe9cba17cee8d8
1223,a48bd739ee695a48974bb3ca2f7c793c,051569be1a5b6f453a41a694f6376b82,051569be1a5b6f453a41a694f6376b82


Как видим модель работает корректно, при этом она может выбрать как track_id самого трека, так и track_id одного из каверов. К сожалению адекватно проверить качество данной модели не получится, так как, во-первых, в первоначальном датасете у большинства треков отсутствовал original_track_id, во-вторых, при работе над второй моделью было установлено, что есть некорректно размеченные данные.

## Выводы

В данном разделе решалась задача поиска исходного трека в цепочке каверов, в ходе решения было сделано следующее:
1. Предложена и реализована следующая логика решения задачи поиска исходного трека в цепочке каверов:
    - сначала проводится анализ цепочки каверов и при отсутствии цепочки как таковой (модель из второй задачи не нашла похожие треки в базе), будем считать сам трек оригиналом;
    - если цепочка треков была обнаружена, то получаем самый ранний из треков на основании года, полученному из isrc (по факту год регистрации трека);
    - если несколько треков соответствуют самому раннему году, либо если есть треки, для которых год неизвестен, то отдаём их для прогноза модели, обученной для решения задачи классификации кавер / оригинал, и по predict_proba выбираем тот, в котором модель больше уверена.
2. Написаны функции, позволяющие на основании прогноза модели для 2 задачи и использую модель для 1 задачи получить датасет как с цепочкой каверов, так и с track_id оригинального трека.